In [ ]:
# Importing necessary modules
import seaborn as sns
import pyspark.sql.functions as F
import pandas as pd
import matplotlib.pyplot as plt
from operator import add
from functools import reduce
import numpy as np
import re
import os
from pyspark.sql.types import StructField, StructType, StringType, LongType
from pyspark.sql.functions import *
import random

# Setting up visualization
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [ ]:
cols_to_keep = [
    "Voters_Gender", # cat
    "Voters_Age", # num
#    "Voters_BirthDate", # ignore
    "Residence_Families_HHCount", # num
    "Residence_HHGender_Description", # cat
    "Mailing_Families_HHCount", # num
    "Mailing_HHGender_Description", # cat

#   !! voter party affiliation
    "Parties_Description", 
    
    # cat
    "CommercialData_PropertyType",
    "AddressDistricts_Change_Changed_CD",
    "AddressDistricts_Change_Changed_SD",
    "AddressDistricts_Change_Changed_HD",
    "AddressDistricts_Change_Changed_County",
    
    "Residence_Addresses_Density", # num
    
    # cat
    "CommercialData_EstimatedHHIncome",
    "CommercialData_ISPSA",
#    "CommercialData_AreaMedianEducationYears",
    "CommercialData_AreaMedianHousingValue",
    "CommercialData_MosaicZ4Global",
#     "CommercialData_AreaPcntHHMarriedCoupleNoChild",  ## Redundant
#     "CommercialData_AreaPcntHHMarriedCoupleWithChild",
#     "CommercialData_AreaPcntHHSpanishSpeaking",
#     "CommercialData_AreaPcntHHWithChildren",
    "CommercialData_StateIncomeDecile",
#    "Ethnic_Description",
    "EthnicGroups_EthnicGroup1Desc",
    "CommercialData_DwellingType",
    "CommercialData_PresenceOfChildrenCode",
#    "CommercialData_PresenceOfPremCredCrdInHome", ## too many missing
    "CommercialData_DonatesToCharityInHome",
    "CommercialData_DwellingUnitSize",
    "CommercialData_ComputerOwnerInHome",
    "CommercialData_DonatesEnvironmentCauseInHome",
    "CommercialData_Education",
    
#   Don't include because of lookahead bias  
#     "Voters_VotingPerformanceEvenYearGeneral",
#     "Voters_VotingPerformanceEvenYearPrimary",
#     "Voters_VotingPerformanceEvenYearGeneralAndPrimary",
#     "Voters_VotingPerformanceMinorElection",
    
#   Other control variables that expect to be highly associated with outcome:
#     "ElectionReturns_P08CountyTurnoutAllRegisteredVoters",
#     "ElectionReturns_P08CountyTurnoutDemocrats",
#     "ElectionReturns_P08CountyTurnoutRepublicans",
    "General_2000",
    "General_2004",
    "PresidentialPrimary_2000",
    "PresidentialPrimary_2004",
        
#   Outcome variable (indiana law happens in 2005, approved by SCOTUS before presidential election in 2008)
    "General_2008"
]

In [ ]:
# These are the states that do not have strict voter ID laws:
#  'VM2Uniform--CA--2021-05-02',	VM2Uniform--CA--2021-05-02	CA	x	California
#  'VM2Uniform--IL--2021-03-05',	VM2Uniform--IL--2021-03-05	IL	x	Illinois
#  'VM2Uniform--MA--2021-01-19',	VM2Uniform--MA--2021-01-19	MA	x	Massachusetts
#  'VM2Uniform--MD--2021-02-15',	VM2Uniform--MD--2021-02-15	MD	x	Maryland
#  'VM2Uniform--ME--2021-05-28',	VM2Uniform--ME--2021-05-28	ME	x	Maine
#  'VM2Uniform--MN--2021-02-14',	VM2Uniform--MN--2021-02-14	MN	x	Minnesota
#  'VM2Uniform--NC--2021-05-18',	VM2Uniform--NC--2021-05-18	NC	x	North Carolina
#  'VM2Uniform--NE--2021-01-20',	VM2Uniform--NE--2021-01-20	NE	x	Nebraska
#  'VM2Uniform--NJ--2021-03-11',	VM2Uniform--NJ--2021-03-11	NJ	x	New Jersey
#  'VM2Uniform--NM--2021-02-25',	VM2Uniform--NM--2021-02-25	NM	x	New Mexico
#  'VM2Uniform--NV--2021-06-13',	VM2Uniform--NV--2021-06-13	NV	x	Nevada
#  'VM2Uniform--NY--2021-03-15',	VM2Uniform--NY--2021-03-15	NY	x	New York
#  'VM2Uniform--OR--2021-02-05',	VM2Uniform--OR--2021-02-05	OR	x	Oregon
#  'VM2Uniform--PA--2021-05-20',	VM2Uniform--PA--2021-05-20	PA	x	Pennsylvania
#  'VM2Uniform--VT--2021-05-28',	VM2Uniform--VT--2021-05-28	VT	x	Vermont

# For each of these states, I want to pull enough samples to get a total sample of 1/2 M; can increase later

# grab files
states =  [
# For now, just exclude New York and Califonria, because the parquet files take too long to read
 'VM2Uniform--VT--2021-05-28',
    
    
    
#   'VM2Uniform--IL--2021-03-05',
#   'VM2Uniform--MA--2021-01-19',
#   'VM2Uniform--MD--2021-02-15',
#   'VM2Uniform--ME--2021-05-28',
#   'VM2Uniform--MN--2021-02-14',
#   'VM2Uniform--NC--2021-05-18',
#   'VM2Uniform--NE--2021-01-20',
#   'VM2Uniform--NJ--2021-03-11',
#   'VM2Uniform--NM--2021-02-25',
#   'VM2Uniform--NV--2021-06-13',
#   'VM2Uniform--OR--2021-02-05',
#   'VM2Uniform--PA--2021-05-20',
 
    
     #'VM2Uniform--CA--2021-05-02',

     #'VM2Uniform--NY--2021-03-15',

]

# bucket file path for all state parquet files
gcs_path = 'gs://pstat135-voter-file/VM2Uniform'

# create list of state abbreviations
pattern = re.compile(r"(?<=--)[A-Z]{2}")
state_abvs = re.findall(pattern, ''.join(states))

# do first iteration
print('VM2Uniform--VT--2021-05-28')

num_per_state = 100000

df_ref = spark.read.parquet("/".join([gcs_path, 'VM2Uniform--VT--2021-05-28']))
df_ref = df_ref.select(cols_to_keep)

numrows = {'VM2Uniform--VT--2021-05-28': df_ref.count()}

print("%d" % (numrows['VM2Uniform--VT--2021-05-28']))
    
percentage_sample = num_per_state / numrows['VM2Uniform--VT--2021-05-28']
    
df_ref = df_ref.sample(True, percentage_sample, seed = 19480384)
df_ref = df_ref.withColumn('STATE', F.lit(state_abvs[0]))
 
next_states = states[1:]

# do the rest of the iterations
for i, one_state in enumerate(next_states):

    print("%s: " % (one_state), end="")
    
    # read dataframe for one_state
    tmp_ref = spark.read.parquet("/".join([gcs_path, one_state]))
    tmp_ref = tmp_ref.select(cols_to_keep)
    numrows[one_state] = tmp_ref.count()
    print("%d" % (numrows[one_state]))
    
    percentage_sample = num_per_state / numrows[one_state]
    
    tmp_ref = tmp_ref.sample(True, percentage_sample, seed = 19480384)
    tmp_ref = tmp_ref.withColumn('STATE', F.lit(state_abvs[i+1]))
    
    df_ref = df_ref.union(tmp_ref)      

# gcs_path = 'gs://pstat135-voter-file/VM2Uniform'
# numrows = dict()
# first DF
#df_ref = spark.read.parquet('gs://pstat135-voter-file/VM2Uniform/')
df_ref.printSchema()
df_ref.count()

In [ ]:
# others
for one_state in states[1:]:
    print("%s: " % (one_state), end="")
    
    # read dataframe for one_state
    df = spark.read.parquet("/".join([gcs_path, one_state]))
    
    # sample small proportion
    sample_df = df.sample(True, percentage_sample, seed = 19480384)
    
    # Then, rbind / concatenate all the files together
    df_ref = df_ref.union(sample_df)

# Save as parquet into google cloud

df_ref.write.format("parquet").save("total_reference_sample")
df_ref = spark.read.parquet("total_reference_sample")

# Then, run the Indiana file's code on them

In [ ]:
# for categorical variables, replace missing value with "O" for 
# other (including General_2000, General_2004, PresidentialPrimary_2000, PresidentialPrimary_2004)

# Get a list of all column types:
numeric_cols = [item[0] for item in df_ref.dtypes if not (item[1].startswith('string') or item[1].startswith('date'))]
categorical_cols = [item[0] for item in df_ref.dtypes if item[1].startswith('string')]

categorical_cols
#numeric_cols

In [ ]:
df_ref.select('CommercialData_DwellingType',
 'CommercialData_PresenceOfChildrenCode',
 'CommercialData_DonatesToCharityInHome',
 'CommercialData_DwellingUnitSize').show()

In [ ]:
numeric_cols = [
    'Voters_Age',
    'Residence_Families_HHCount',
    'Mailing_Families_HHCount',
    'Residence_Addresses_Density'
]

trinary_cols = [
    'CommercialData_DonatesToCharityInHome',
    'CommercialData_ComputerOwnerInHome',
    'CommercialData_DonatesEnvironmentCauseInHome'
]

binary_cols = [
    "General_2000",
    "General_2004",
    "PresidentialPrimary_2000",
    "PresidentialPrimary_2004"
]

other_cols = [c for c in df_ref.columns if c != "General_2008"]
other_cols = [c for c in other_cols if c not in (numeric_cols + trinary_cols + binary_cols)]

categorical_cols = other_cols + binary_cols + trinary_cols

In [ ]:
for c in numeric_cols:
    df_ref = df_ref.withColumn(c, F.col(c).cast("float").alias(c))
df_ref = df_ref.fillna("U", subset= trinary_cols)
df_ref = df_ref.fillna("Missing", subset = other_cols)
df_ref = df_ref.fillna("N", subset = binary_cols)
df_ref.printSchema()

In [ ]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import StringIndexer

# Create copy of working df
new_df = df_ref.alias('new_df')

# Impute the missing values in the numerical columns with the mean -- minimize change to z-scores of given data
imputer = Imputer(
    inputCols=numeric_cols, 
    outputCols=["{}_imputed".format(c) for c in numeric_cols]
)

new_df = imputer.fit(new_df).transform(new_df)

# Impute categorical columns -- maybe it's better to drop these records
# new_df = new_df.fillna("missing", subset = categorical_cols)

# Assemble numerical columns into vector
vec = VectorAssembler(inputCols = [c+"_imputed" for c in numeric_cols], outputCol = "numFeatures")
new_df = vec.transform(new_df)

# Standardize the numeric columns
scaler = StandardScaler(inputCol="numFeatures", outputCol="scaledFeatures",
                        withStd=True, withMean=True)
new_df = scaler.fit(new_df).transform(new_df)

# output columns to be indexed
indexed_cols = [f"index{i}" for i in range(len(categorical_cols))]

# Ecode categorical variables
indexer = StringIndexer(inputCols = categorical_cols, outputCols = indexed_cols)
new_df = indexer.fit(new_df).transform(new_df)

# Through categorical variables in their own vector
vec = VectorAssembler(inputCols = indexed_cols, outputCol = "catFeatures")
new_df = vec.transform(new_df)

# Concatenate scaled numerical and categorical vectors into single features vector
vec = VectorAssembler(inputCols = ['scaledFeatures', 'catFeatures'], outputCol = "features")
new_df = vec.transform(new_df)

new_df.select("features").show()

In [ ]:
new_df.groupBy('General_2004').count().show()

In [ ]:
indi.groupBy("General_2004").count().show()

In [ ]:
2734476/(2734476 + 1662100)

In [ ]:
336427/(162862+336427)

In [ ]:
from pyspark.ml.functions import vector_to_array
from pyspark.sql.types import FloatType
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.feature import RFormula

indi = spark.read.parquet("gs://voter-project-235-25/VM2Uniform--IN--2021-01-15_parq")
indi = indi.sample(True, 0.1, seed = 19480384)

# clean the total parquet file--------------

num_voters_indiana = indi.count()
num_voters_not_indiana = df_ref.count()
pct_sample = num_voters_indiana / num_voters_not_indiana

# empty list to store the estimated average treatment effects:
ATEs = []

# empty dictionary to store output
stored_DF = {}

i = 0
for i in range(1000):
    i = i + 1 # from 1 - 100 rather than 0 to 100

    # take random sample of the total parquet file (equivalent to the size of indiana)
    df_ref = df_ref.sample(True, pct_sample, seed = i)

    # create dummy DATA:
    indi = indi.withColumn('Voters_Age', rand())
    df_ref = df_ref.withColumn('Voters_Age', rand())
    
    indi = indi.select(["Voters_Age"])
    df_ref = df_ref.select(["Voters_Age"])

    indi = indi.withColumn('Voters_Age', col('Voters_Age').cast('double'))
    df_ref = df_ref.withColumn('Voters_Age', col('Voters_Age').cast('double'))

    indi = indi.withColumn('General_2008_RANDOM', when(rand() > 0.5, 1).otherwise(0))
    df_ref = df_ref.withColumn('General_2008_RANDOM', when(rand() > 0.5, 1).otherwise(0))

    # create a column with "LAW == 0" for non-Indiana states
    df_ref = df_ref.withColumn("LAW", lit(0))

    # create a column with "LAW == 1" for Indiana
    indi = indi.withColumn("LAW", lit(1))

    # union the two together
    df = df_ref.union(indi)

    df_input_logistic = df.drop("General_2008_RANDOM")

    # fit logistic model on the intervention (variable Law)
    nrow_df_input_logistic_start_check = df_input_logistic.count()
    df_input_logistic = df_input_logistic.fillna(0)
    df_input_logistic.columns
    supervised = RFormula(formula="LAW ~ .")
    fittedRF = supervised.fit(df_input_logistic) # inspect column types
    prepareddf_input_logistic = fittedRF.transform(df_input_logistic) # create feature and label columns
    prepareddf_input_logistic.show(5, truncate = False)
    lr = LogisticRegression(labelCol="label",featuresCol="features")
    print(lr.explainParams())
    lrModel = lr.fit(prepareddf_input_logistic) # train model
    lrModel.transform(prepareddf_input_logistic).select("label", "prediction") # fitted values
    plt.rcParams["figure.figsize"] = (8,6)
    beta = np.sort(lrModel.coefficients)
    plt.plot(beta)
    plt.ylabel('Beta Coefficients')

    # get a propensity score from the probability as a new column:
    fitted = lrModel.transform(prepareddf_input_logistic)
    fitted = fitted.withColumn('probability', vector_to_array('probability'))
    array_mean = udf(lambda x: float(np.mean(x)), FloatType())
    fitted = fitted.withColumn("propensity_score", array_mean("probability"))

    fitted.count() == nrow_df_input_logistic_start_check

    # new column that called weight that is T - PS / (PS * 1 - PS)
    fitted = fitted.withColumn("weight", (col("label") - col("propensity_score")) / (col("propensity_score") * (1-col("propensity_score"))))

    # merge back in the 2008 general election OUTCOME data
    fitted = fitted.withColumn("row_id", monotonically_increasing_id())
    df = df.withColumn("row_id", monotonically_increasing_id())
    num_row_df_prior = df.count()
    df = df.join(fitted, ["row_id", "Voters_Age", "LAW"]).drop("row_id")
    num_row_df_prior == df.count()

    # calculate the weighted average
    df = df.withColumn("weighted_outcome", col("General_2008_RANDOM") * col("weight"))

    # store weighted average into list
    ATE_this_round = df.agg(avg(col("weighted_outcome"))).collect()[0][0]
    ATEs.append(ATE_this_round)

    # store the DataFrame into a dictionary
    stored_DF[f"{i}"] = df

In [ ]:
stored_DF

In [ ]:
ATEs

In [ ]:
df.show(10)

In [ ]:
fitted.show(10)

In [ ]:
# create box plot showing propensity score by treated or not treated
# divy this vizualization up by states (?)

# sns.boxplot(x="success_expect", y="propensity_score", data=data_ps)
# plt.title("Confounding Evidence")